# Assignment 1

1. Реализовать Блум фильтр с помощью битового массива.
Например, вы можете использовать [Битовые операции](https://wiki.python.org/moin/BitwiseOperators) или библиотеку bitarray.

2. Провести численный эксперимент при false postive rate = 0.05, и количестве объектов S = 1 000 000.
Убедится, полученные на семинаре оптимальные параметры фильтра позволяют достичь заявленного false positive rate.
Посчитать  $\frac {\epsilon - \hat \epsilon} \epsilon$, где $\hat \epsilon$ - ваша экспериментальная оценка false positive rate. В качестве объектов используйте строки.

## Фильтр Блума

In [1]:
import random
import math
from bitarray import bitarray 
from sympy.ntheory.generate import prime

In [57]:
class BloomFilter(object):
    
    def __init__(self, false_positive, S):
        self.a_size = math.ceil(-(S * math.log(false_positive))/(math.log(2)**2))
        self.k_func = math.ceil((self.a_size/S) * math.log(2))
        #self.a_size = int(-(S * math.log(false_positive))/(math.log(2)**2))
        #self.k_func = int((self.a_size/S) * math.log(2))
        self.array = bitarray(self.a_size)
        self.array.setall(0)
        self.hash_p = []
        for i in range (self.k_func):
            nth = random.randint(0, self.a_size)
            p = prime(1000+random.randint(1,10)*nth)
            self.hash_p.append((p, random.randint(1, p-1)))
        
    def add(self, value):
        for x in self.multi_hash(value):
            self.array[x] = 1
    
    def check(self, value):
        for x in self.multi_hash(value):
            if self.array[x] == 0:
                return False
        return True
    
    def _hash(self, value, p, x):
        for p, x in self.hash_p:
            h = 0
            for i in range(len(value)-1, -1, -1):
                h = (h*x + ord(value[i]) + p) % p
        return h
                
    def multi_hash(self, value):
        for p, x in self.hash_p:
            h = self._hash(value, p, x)
            yield h % self.a_size

## Test

In [4]:
import string

In [18]:
''.join(random.choices(string.ascii_uppercase + string.digits+string.ascii_lowercase, k=random.randint(1,50)))

'12Cn2RxxhSxrxGiVvv7Gf4l'

In [62]:
def test_bloomfilter(p, n):
    bf = BloomFilter(p, n)
    print("{} bit array".format(bf.a_size)) 
    print("{} hash functions".format(bf.k_func)) 
    train = set(''.join(random.choices(string.ascii_uppercase + string.digits+string.ascii_lowercase, k=random.randint(1,50))) for _ in range(n))
    for i in train:
        bf.add(i)
    TP, FP, TN, FN = 0, 0, 0, 0
    for _ in range(n):
        value = ''.join(random.choices(string.ascii_uppercase + string.digits+string.ascii_lowercase, k=random.randint(1,50)))
        res = bf.check(value)
        if res is True:
            if value in train: TP += 1
            else: FP += 1
        else:
            if value in train: FN += 1
            else: TN += 1
    TP = TP / n
    FP = FP / n
    TN = TN / n
    FN = FN / n
    print ('TP={}, FP={}, TN={}, FN={}'.format(TP, FP, TN, FN))
    return FP/(FP+TN)

In [ ]:
test_bloomfilter(0.05, 10**6)